In [ ]:
from xml.dom import minidom
import pandas as pd
import numpy as np
import os
import glob
import time
from datetime import datetime
from datetime import timedelta
import gzip
import mechanicalsoup
import urllib
import requests
from bs4 import BeautifulSoup
import json
import time
from langdetect import detect

In [ ]:
def get_wos_data(file):
    soup = BeautifulSoup(open(file, encoding='utf-8'), 'html.parser')
    
    # Title
    title = str(soup.find("div", {"class": "title"}))
    title = title[title.find('<value>') + 7:title.find('</value>')]
    
    # Authors
    authors = []
    authors_array = soup.findAll("a", {"title": "Find more records by this author"})

    for author in authors_array:
        author = str(author)
        authors.append(author[author.find('>', 400)+1:author.rfind('</a>')] + '.')
    
    # Keywords
    a_array = soup.findAll("a", {"class": "snowplow-author-keyword-link"})
    keywords = []

    for a in a_array:
        a = str(a)
        keywords.append(a[a.find('>', 450) + 1:-4])
    
    # Abstract and Date
    p_array = soup.findAll("p", {"class": "FR_field"})
    for p in p_array:
        p = str(p)
        if p[19] == '>' and (p[20] != '\n' and p[20] != ' '):
            abstract = p[p.find('>') + 1:-4]
        elif p.startswith('<p class="FR_field">\n<span class="FR_label">Published:</span>'):
            date = p[p.find('<value>') + 7:p.find('</value>')]
    
    # No. of citations
    h2 = str(soup.find('h2'))
    citations = h2[h2.find(':') + 2:-5]
    
    # Funding agency
    tr = soup.find("tr", {"class": "fr_data_row"})
    funding = tr.find("td").string[:-1]
    
    data = {}
    data['title'] = title
    data['authors'] = authors
    data['keywords'] = keywords
    data['date'] = date
    data['abstract'] = abstract
    data['language'] = detect(abstract)
    data['citations'] = citations
    data['funding_agency'] = funding
    
    json_data = json.dumps(data)
    return json_data


def get_newest_file(path):
    files = os.listdir(path)
    paths = [os.path.join(path, basename) for basename in files]
    return max(paths, key=os.path.getctime)

def get_url(DOI):
    start_url = 'http://apps.webofknowledge.com/WOS_GeneralSearch_input.do?product=WOS&search_mode=GeneralSearch&SID=E4nAHEnmQkjgeXGHVKC&preferencesSaved='
    
    browser = mechanicalsoup.StatefulBrowser()
    browser.open(start_url)
    browser.select_form('form[action="/WOS_GeneralSearch.do"]')
    # browser.get_current_form().print_summary()
    browser['value(input1)'] = DOI
    response = browser.submit_selected()

    start = response.text.find('href=\"/full_record') + 6
    end = response.text.find('>', start) - 1
    
    url = ''
    if response.text[start:end].startswith('/full_record'):
        url = 'http://apps.webofknowledge.com' + response.text[start:end]
        url = url.replace('amp;', '')
    print(url)
    return url

Iz XML datoteke, ki smo jo dobili iz Sicrisa, izvlečemo DOI člankov, ki ga bomo uporabili za iskanje člankov po portalu Web of Scence

In [ ]:
xmldoc = minidom.parse('downloads/cobiss-org/2019-06-24-144940_cobiss.xml')
listDOI = xmldoc.getElementsByTagName('DOI')
listWOS = xmldoc.getElementsByTagName('WoS')
DOIs = []
for doi, wos in zip(listDOI, listWOS):
    if len(doi.childNodes) != 0 and len(wos.childNodes) != 0:
        DOIs.append([doi.childNodes[0].nodeValue, wos.childNodes[0].nodeValue[-15:]])
        
DOIs;

In [ ]:
for DOI in DOIs:
    if not os.path.exists('downloads/wos/{:}'.format(DOI[1])):
        os.mkdir('downloads/wos/{:}'.format(DOI[1]))

    if len(os.listdir('downloads/wos/{:}'.format(DOI[1]))) != 0:
        file = get_newest_file('downloads/wos/{:}'.format(DOI[1]))
        file_date = datetime.strptime(file[file.find('\\')+1:-5], '%Y-%m-%d')
        if file_date < now - timedelta(days = 14):
            url = get_url(DOI[0])
            if len(url):
                response = requests.get(url)
                file = open('downloads/wos/{:}/{:}.html'.format(DOI[1], now.strftime('%Y-%m-%d')), "w", encoding='utf-8')
                file.write(response.text)
                file.close()
    else:
        url = get_url(DOI[0])
        if len(url):
            response = requests.get(url)
            file = open('downloads/wos/{:}/{:}.html'.format(DOI[1], now.strftime('%Y-%m-%d')), "w", encoding='utf-8')
            file.write(response.text)
            file.close()